In [1]:
from aind_data_access_api.document_db import MetadataDbClient
import os
import pandas as pd

In [2]:
DOC_DB_HOST = os.getenv("CUSTOM_KEY_3")
DOC_DB_DATABASE = "metadata"
DOC_DB_COLLECTION = "data_assets"

doc_db_client = MetadataDbClient(host=DOC_DB_HOST, database=DOC_DB_DATABASE, collection=DOC_DB_COLLECTION,)
results = doc_db_client.retrieve_data_asset_records(filter_query=None, projection={"_name":1, "_created": 1, "_location":1, "data_description.data_level": 1}, paginate=False)

In [3]:
print(results[5])

has_data_desc = []


data_levels = {}
for result in results:
    try:
        if result.data_description['data_level'] not in data_levels:
            data_levels[result.data_description['data_level']] = 1
        else:
            data_levels[result.data_description['data_level']] += 1
            
        has_data_desc.append(result)
    except:
        continue
            
data_levels

id='7777c7bc-25ed-46c1-9359-1040738a3dc5' name='SmartSPIM_717442_2024-03-05_02-22-06' created=datetime.datetime(2024, 3, 31, 21, 20, 49, tzinfo=TzInfo(UTC)) location='s3://aind-open-data/SmartSPIM_717442_2024-03-05_02-22-06' data_description={'data_level': 'raw'}


{'derived': 777, 'raw': 1430, 'raw data': 2851, 'derived data': 834}

In [4]:
old_tags = [result._name for result in has_data_desc if result.data_description['data_level'] == 'raw' or result.data_description['data_level'] == 'derived']
print(old_tags)

['SmartSPIM_719651_2024-03-08_22-26-57_stitched_2024-04-01_10-14-17', 'SmartSPIM_717442_2024-03-05_02-22-06_stitched_2024-04-01_09-27-09', 'SmartSPIM_721826_2024-03-18_22-41-49_stitched_2024-04-01_08-52-57', 'SmartSPIM_719651_2024-03-08_22-26-57', 'SmartSPIM_720811_2024-03-18_19-42-55', 'SmartSPIM_717442_2024-03-05_02-22-06', 'SmartSPIM_721826_2024-03-18_22-41-49', 'ecephys_716465_2024-02-14_13-38-32_sorted_2024-03-29_11-29-26', 'ecephys_686740_2023-10-24_12-46-24_sorted_2024-03-29_15-31-45', 'ecephys_664851_2023-11-13_12-49-51_sorted_2024-03-29_15-31-32', 'ecephys_664851_2023-11-14_12-44-33_sorted_2024-03-29_15-31-22', 'ecephys_686176_2023-12-06_13-03-34_sorted_2024-03-29_15-30-49', 'ecephys_676909_2023-12-11_14-24-35_sorted_2024-03-29_11-29-39', 'SmartSPIM_710626_2024-03-29_14-55-52_stitched_2024-03-31_10-33-05', 'ecephys_717120_2024-03-06_12-54-27_sorted-opto_2024-03-30_11-06-49', 'ecephys_713854_2024-03-08_15-43-01_sorted-opto_2024-03-30_10-53-22', 'SmartSPIM_709140_2024-03-29_14-3

In [5]:
whodunnit = set([result.split('_')[0] for result in old_tags])
print(whodunnit)

{'SmartSPIM', 'multiplane-ophys', 'test', 'behavior', 'HCR', 'FIP', 'exaSPIM', 'confocal', 'single-plane-ophys', 'ecephys'}


In [6]:
df = pd.DataFrame(columns=['asset_name','tags'])
for result in has_data_desc:
    if result.data_description['data_level'] == 'raw' or result.data_description['data_level'] == 'derived':
        df = pd.concat([pd.DataFrame([[result._name, result.data_description['data_level']]], columns=df.columns), df], ignore_index=True)
    
df

,asset_name,tags
0,exaSPIM_3163606_2023-11-17_12-54-51,raw
1,SmartSPIM_708396_2024-01-08_11-11-54_stitched_...,derived
2,SmartSPIM_691386_2023-08-31_16-53-18_stitched_...,derived
3,behavior_123456_2023-10-04_04-00-00,raw
4,exaSPIM_659146_2023-11-10_14-02-06,raw
...,...,...
2202,SmartSPIM_720811_2024-03-18_19-42-55,raw
2203,SmartSPIM_719651_2024-03-08_22-26-57,raw
2204,SmartSPIM_721826_2024-03-18_22-41-49_stitched_...,derived
2205,SmartSPIM_717442_2024-03-05_02-22-06_stitched_...,derived


In [7]:
results_file_path = "/results/old_tags.txt"

with open(results_file_path, 'a') as f:
    df_string = df.to_string(header=False, index=False)
    f.write(df_string)